# Display results POET AI Surrogate ML framework tests

In [10]:
import csv
# function to get EMA data for conducted experiments
def get_EMA_data(PID):
    power = {
        "load_data": {"GPU": 0, "CPU": 0},
        "setup": {"GPU": 0, "CPU": 0},
        "process_data": {"GPU": 0, "CPU": 0}
        }
    time = {
        "load_data": 0,
        "setup": 0,
        "process_data": 0
        }
    with open("output.EMA." + PID) as file:
        csv_reader = csv.reader(file, delimiter =",")
        next(csv_reader) # skip header row
        for i, row in enumerate(csv_reader):
            if row[6] == "NVIDIA A2":
                device = "GPU"
            elif "CPU-" in row[6]:
                device = "CPU"
            else:
                print("UNKNOWN EMA DEVICE!")
            power[row[1]][device] += int(row[7])
            if time[row[1]] == 0: # time is only taken from first device
                time[row[1]] = int(row[8])
        return [power, time]
    
def get_csv_data(PID, data_type):
    try: # GPU files ony exist for jobs on GPU nodes, training history only for train jobs 
        with open("output." + data_type + "." + PID) as file:
            csv_reader = csv.reader(file, delimiter =",")
            return next(csv_reader)
    except OSError:
        return []

In [11]:
# read parameters and results of conducted experiments
experiments = {}
with open("experiments.csv") as file:
    csv_reader = csv.reader(file, delimiter =",")
    next(csv_reader) # skip header row
    for row in csv_reader:
        ema_data = get_EMA_data(row[4])
        experiments[row[4]] = {
            "framework": row[0],
            "task": row[1],
            "device": row[5],
            "model": row[2],
            "batch_size": row[3],
            "power": ema_data[0], # in micro joules
            "time": ema_data[1], # in micro seconds
            "GPU_usage": get_csv_data(row[4], "GPU_usage"),  # percentage in 100 ms intervals
            "training_val_loss": get_csv_data(row[4], "training_history"),
            }

In [15]:
## Runtime by framework
for pid, data in experiments.items():
    print(pid, data["task"], data["power"])

24899 train {'load_data': {'GPU': 3132000, 'CPU': 32010624}, 'setup': {'GPU': 164166000, 'CPU': 1310787565}, 'process_data': {'GPU': 9275111000, 'CPU': 22877094652}}
12904 train {'load_data': {'GPU': 7558000, 'CPU': 32772630}, 'setup': {'GPU': 216262000, 'CPU': 1309484979}, 'process_data': {'GPU': 31433618000, 'CPU': 46774561064}}
975 predict {'load_data': {'GPU': 5140000, 'CPU': 20290999}, 'setup': {'GPU': 263625000, 'CPU': 1311740406}, 'process_data': {'GPU': 926766000, 'CPU': 2105519803}}
1222 predict {'load_data': {'GPU': 2033000, 'CPU': 21375257}, 'setup': {'GPU': 158058000, 'CPU': 1299897812}, 'process_data': {'GPU': 1598947000, 'CPU': 4341897009}}
13881 train {'load_data': {'GPU': 0, 'CPU': 33968802}, 'setup': {'GPU': 0, 'CPU': 1089064899}, 'process_data': {'GPU': 0, 'CPU': 65411063883}}
3841 predict {'load_data': {'GPU': 0, 'CPU': 0}, 'setup': {'GPU': 0, 'CPU': 0}, 'process_data': {'GPU': 0, 'CPU': 0}}
4686 predict {'load_data': {'GPU': 0, 'CPU': 0}, 'setup': {'GPU': 0, 'CPU': 